In [1]:
import requests
import pandas as pd
from API_Key import api_key

pages = list(range(1,11))

popular = pd.DataFrame()

for page in pages:
    movieData = pd.DataFrame(requests.get(f'https://api.themoviedb.org/3/movie/popular?api_key={api_key}&language=en-US&page={page}').json()['results'])
    popular = popular.append(movieData)

popular.head()


popular = popular[popular['adult'] == False]

titleNid = popular[['id','title']]

titleNid = titleNid.reset_index(drop = True)

/var/folders/61/jyjwflp16vv8tst5lfm0g4xr0000gn/T/ipykernel_7040/4079447556.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  popular_features = popular.drop(['adult', 'backdrop_path', 'id', 'original_title', 'overview', 'poster_path', 'title','video'],1)


In [7]:
popular_features = popular[['id','title','release_date','popularity','vote_average','vote_count']]
popular_features = popular_features.rename(columns={
                                        "id":"movie_id",
                                        "popularity":"movie_popularity"
                                        })

In [12]:
titleNid = titleNid.rename(columns={"id" : 'movie_id'})

In [13]:
titleNid.to_csv('Tables/titleNid.csv',index = False)

In [3]:
# Loop through ids for revnue values
id_list = titleNid['movie_id'].tolist()

revenue_list = []
budget_list = []

for movie in id_list:
    x = requests.get(f'https://api.themoviedb.org/3/movie/{movie}?api_key={api_key}&language=en-US').json()['revenue']
    y = requests.get(f'https://api.themoviedb.org/3/movie/{movie}?api_key={api_key}&language=en-US').json()['budget']
    revenue_list.append(x)
    budget_list.append(y)

revenue_df = pd.DataFrame(
    {'movie_id':id_list,
    'budget': budget_list,
    'revenue':revenue_list
    })

revenue_df.reset_index(drop = True)

,movie_id,budget,revenue
0,566525,150000000,430238384
1,370172,242000000,708000000
2,580489,110000000,442000000
3,522402,0,0
4,512195,200000000,0
...,...,...,...
195,324786,40000000,175302354
196,410113,0,0
197,623135,0,0
198,874209,0,0


In [6]:
popular_features

,genre_ids,original_language,popularity,release_date,vote_average,vote_count
0,"[28, 12, 14]",en,8304.971,2021-09-01,7.9,3094
1,"[12, 28, 53]",en,4398.328,2021-09-29,7.6,1870
2,"[878, 28, 12]",en,4128.057,2021-09-30,6.8,1987
3,"[878, 18, 12]",en,3098.869,2021-11-04,8.2,986
4,"[28, 35, 80, 53]",en,3901.386,2021-11-04,6.9,960
...,...,...,...,...,...,...
195,"[18, 36, 10752]",en,197.352,2016-10-07,8.2,10180
196,"[16, 10751, 35, 10402, 14]",fr,173.402,2021-08-20,8.4,86
197,"[35, 18]",ko,176.414,2021-08-11,7.2,23
198,[],en,190.281,2016-09-23,4.9,4


In [9]:

Movie_Data = popular_features.merge(revenue_df, on='movie_id', how='inner')
Movie_Data.to_csv('Tables/Movie_Data.csv', index=False)